In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import log_loss
import category_encoders as ce
from sklearn.neural_network import MLPRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import silhouette_score
from scipy.stats import gaussian_kde
# from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
# from scipy.stats import zscore
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, log_loss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
# from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from umap import UMAP
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import entropy
from sklearn.metrics import precision_recall_curve
from sklearn.manifold import TSNE
from category_encoders import TargetEncoder
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from tqdm import tqdm
from category_encoders import TargetEncoder
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import category_encoders as ce
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import OneClassSVM
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import f_classif
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import FunctionTransformer
from scipy.sparse import hstack, csr_matrix
from category_encoders import CountEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from scipy import sparse
from scipy.optimize import minimize
# from keras.callbacks import ReduceLROnPlateau
# from sklearn.ensemble import VotingAnomalyDetector
from sklearn.mixture import GaussianMixture
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
import networkx as nx
from networkx.algorithms import community

In [2]:
df = pd.read_csv("DATA.csv", sep=';')

In [43]:
df = pd.read_csv("DATA2.csv", sep=';', encoding='latin1')


In [3]:
# ----- Isolation Forest Modifié pour Silhouette Score -----
class IsolationTree:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.tree = None
    @staticmethod
    def _c(n):
        if n <= 1: 
            return 0
        return 2 * (np.log(n - 1) + 0.5772) - (2 * (n - 1) / n)
    def fit(self, X, depth=0):
        if depth >= self.max_depth or len(X) <= 1:
            return {"size": len(X)}
        n_features = X.shape[1]
        feature = np.random.randint(0, n_features)
        min_val, max_val = np.min(X.iloc[:, feature]), np.max(X.iloc[:, feature])
        if min_val == max_val:
            return {"size": len(X)}
        split = np.random.uniform(min_val, max_val)
        left = X[X.iloc[:, feature] < split]
        right = X[X.iloc[:, feature] >= split]

        return {
            "feature": feature,
            "split": split,
            "left": self.fit(left, depth + 1),
            "right": self.fit(right, depth + 1)
        }
    
    def path_length(self, x, node=None, depth=0):
        if node is None:
            node = self.tree
        if "size" in node:
            return depth + self._c(node["size"])
        
        # Make sure we don't try to access non-existent features
        if node["feature"] >= len(x):
            return depth + self._c(1)  # Return a base case if feature is out of bounds
            
        value = x[node["feature"]]

        if value < node["split"]:
            return self.path_length(x, node["left"], depth + 1)
        else:
            return self.path_length(x, node["right"], depth + 1)
    

class IsolationForestScratch:
    def __init__(self, n_trees=100, max_depth=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.trees = []
        self.X = None
        self.threshold = None
        self.scores = None
        self.contamination = 0.05  # Default contamination rate
    

    def anomaly_score(self, X=None):
        X = self.X if X is None else (X.values if isinstance(X, pd.DataFrame) else X)
        scores = []
        
        for i in range(len(X)):
            x = X[i]
            if isinstance(x, pd.Series):  # Convert Series to numpy array if needed
                x = x.values
                
            lengths = [t.path_length(x) for t in self.trees]
            avg = np.mean(lengths)
            score = 2 ** (-avg / IsolationTree._c(len(self.X)))  # Call through IsolationTree
            scores.append(score)
        return np.array(scores)
    def fit(self, X):
        # Convert to numpy array and ensure 2D
        self.X = np.asarray(X)
        if len(self.X.shape) == 1:
            self.X = self.X.reshape(-1, 1)
            
        height_limit = int(np.ceil(np.log2(len(self.X))))
        self.trees = []
        
        for _ in range(self.n_trees):
            sample_idx = np.random.choice(len(self.X), len(self.X) // 2, replace=False)
            sample = self.X[sample_idx]  # Use numpy array directly
            tree = IsolationTree(self.max_depth or height_limit)
            tree.tree = tree.fit(pd.DataFrame(sample))  # Convert to DataFrame only for the tree fitting
            self.trees.append(tree)
        
        # Calculate threshold based on contamination
        scores = self.anomaly_score()
        self.threshold = np.percentile(scores, 100 * (1 - self.contamination))
    
    def get_distance_matrix(self):
        """Retourne une matrice de distance basée sur les scores d'anomalie"""
        scores = self.anomaly_score()
        return cdist(scores.reshape(-1, 1), scores.reshape(-1, 1))
    
    def get_cluster_labels(self, X, threshold=None):
        """
        Returns cluster labels (0=normal, 1=anomaly) based on anomaly scores
        If no threshold is provided, uses the one calculated during fit()
        """
        scores = self.anomaly_score(X)
        threshold = threshold or self.threshold
        
        if threshold is None:
            raise ValueError("Threshold not set. Call fit() first or provide a threshold.")
            
        return (scores > threshold).astype(int)


# ----- LOF Modifié pour Silhouette Score -----
class LOF:
    def __init__(self, k=10):
        self.k = k
        self.X = None
        self.distances = None
        self.threshold = 1.5 

    def fit(self, X, contamination=0.05):
        self.X = X.values if isinstance(X, pd.DataFrame) else X
        self.distances = cdist(self.X, self.X)
        
        # Calculate threshold based on contamination
        scores = self.anomaly_score()
        self.threshold = np.percentile(scores, 100 * (1 - contamination))

    def _reach_dist(self, i, j):
        dist = self.distances[i, j]
        k_dist_j = np.partition(self.distances[j], self.k)[self.k]
        return max(dist, k_dist_j)

    def _lrd(self, i):
        neighbors = np.argsort(self.distances[i])[1:self.k+1]
        reach_dists = [self._reach_dist(i, j) for j in neighbors]
        return 1 / (np.mean(reach_dists) + 1e-10)

    def anomaly_score(self, X=None):
        if X is None:
            X = self.X
        else:
            X = X.values if isinstance(X, pd.DataFrame) else X
            distances = cdist(X, self.X)
        
        lrd_scores_train = [self._lrd(i) for i in range(len(self.X))]
        scores = []
        
        for i in range(len(X)):
            if X is self.X:
                neighbors = np.argsort(self.distances[i])[1:self.k+1]
            else:
                neighbors = np.argsort(distances[i])[:self.k]
            
            lrd_x = 1 / (np.mean([
                max(distances[i][j] if X is not self.X else self.distances[i][j], 
                np.partition(self.distances[j], self.k)[self.k])
                for j in neighbors
            ]) + 1e-10)
            
            ratios = [lrd_scores_train[j] / lrd_x for j in neighbors]
            scores.append(np.mean(ratios))
        
        return np.array(scores)
        
    
    def get_distance_matrix(self):
        """Retourne la matrice de distance originale"""
        return self.distances
    
    def get_cluster_labels(self, X, threshold=None):
        """Return labels for input data X"""
        scores = self.anomaly_score(X)
        threshold = threshold or self.threshold
        return (scores > threshold).astype(int)




# ----- One-Class SVM Simplifié -----
class OneClassSVM_RBF:
    def __init__(self, gamma=0.1):
        self.gamma = gamma

    def fit(self, X):
        self.X_train = X
        self.kernel = self._rbf_kernel(X, X)

    def _rbf_kernel(self, X1, X2):
        sq_dists = cdist(X1, X2, 'sqeuclidean')
        return np.exp(-self.gamma * sq_dists)

    def decision_function(self, X):
        K = self._rbf_kernel(X, self.X_train)
        return np.mean(K, axis=1)

    def anomaly_score(self, X):
        return -self.decision_function(X)

#------Autoencoder------
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=10):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z), z

    
    def fit(self, X):
        # === 1. Vérification et conversion des données ===
        if isinstance(X, pd.DataFrame):
            X = X.values.astype(np.float32)
        elif not isinstance(X, np.ndarray):
            raise ValueError("Input must be DataFrame or numpy array")
        
        # === 2. Initialisation du modèle ===
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        X_tensor = torch.FloatTensor(X).to(device)
        dataset = TensorDataset(X_tensor)
        loader = DataLoader(dataset, batch_size=64, shuffle=True)
        
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        
        # === 3. Entraînement ===
        self.train()
        for epoch in range(30):
            epoch_loss = 0
            for batch in loader:
                x_batch = batch[0]
                optimizer.zero_grad()
                x_recon, _ = self(x_batch)
                loss = criterion(x_recon, x_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            
            if epoch % 10 == 0:
                avg_loss = epoch_loss / len(loader)
                print(f"Epoch {epoch} - Loss: {avg_loss:.4f}")
        
        # === 4. Encodage dans l'espace latent ===
        with torch.no_grad():
            self.eval()
            _, latent = self(X_tensor)
            latent_np = latent.cpu().numpy()
        
        # === 5. Clustering ===
        kmeans = KMeans(n_clusters=2, random_state=42)
        y_pred = kmeans.fit_predict(latent_np)
        print(f" Kmeans Silhouette Score: {silhouette_score(latent_np, y_pred):.3f}")
        # LOF
        lof = LOF(k=20)
        lof.fit(latent_np)
        lof_scores = lof.anomaly_score()
        lof_labels = lof.get_cluster_labels(pd.DataFrame(latent_np))
        print(f"LOF Silhouette Score: {silhouette_score(lof.get_distance_matrix(), lof_labels, metric='precomputed'):.3f}")
        
        # Isolation Forest
        iso_forest = IsolationForestScratch(n_trees=100)
        iso_forest.fit(pd.DataFrame(latent_np))
        iso_scores = iso_forest.anomaly_score(pd.DataFrame(latent_np))
        iso_labels = iso_forest.get_cluster_labels(pd.DataFrame(latent_np))
        print(f"Isolation Forest Silhouette Score: {silhouette_score(iso_forest.get_distance_matrix(), iso_labels, metric='precomputed'):.3f}")
        self.latent_np = latent_np
        self.lof = lof
        self.iforest = iso_forest
        self.lof_labels = lof_labels
        self.iso_labels = iso_labels
        return {
            'latent': pd.DataFrame(latent_np),
            'cluster_labels': y_pred,
            'lof_scores': lof_scores,
            'iso_scores': iso_scores,
            'iso_labels': iso_labels,
            'lof_labels': lof_labels
        }
    def encode(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values.astype(np.float32)
        elif not isinstance(X, np.ndarray):
            raise ValueError("Input must be DataFrame or numpy array")

        self.eval()
        device = next(self.parameters()).device

        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(device)
            latent = self.encoder(X_tensor)
            return latent.cpu().numpy()

    def reconstruct(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values.astype(np.float32)

        self.eval()
        device = next(self.parameters()).device
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(device)
            X_recon = self.decoder(self.encoder(X_tensor))
            return X_recon.cpu().numpy()

#--------META MODELE------
class BayesianOrdinalEncoder:
    def __init__(self, cols):
        self.cols = cols
        self.encoder = None

    def fit_transform(self, X, y):
        X_ = X.copy()
        self.encoder = ce.LeaveOneOutEncoder(cols=self.cols, random_state=42, sigma=0.1)
        return self.encoder.fit_transform(X_, y)

    def transform(self, X):
        X_ = X.copy()
        return self.encoder.transform(X_)
class ShrinkageBoostingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=5, stack_model=None, fast_mode=True):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.tree_weights = []
        self.scaler = StandardScaler()
        self.encoder = None
        
        self.fast_mode = fast_mode
        
        if stack_model is None:
            from sklearn.ensemble import RandomForestClassifier
            self.stack_model = RandomForestClassifier(n_estimators=100, random_state=42)
        else:
            self.stack_model = stack_model

    def _preprocess(self, X):
        X = X.copy()
        for col in X.columns:
            col_str = str(col)
            if 'date' in col_str.lower():
                try:
                    X[col] = pd.to_datetime(X[col], errors='coerce').astype('int64') // 10**9
                except Exception:
                    pass
        
        for col in X.columns:
            if X[col].dtype == 'object':
                X[col] = X[col].fillna('missing')
            else:
                X[col] = X[col].fillna(X[col].median())
        return X

    def fit(self, X, y):
        X = pd.DataFrame(X).reset_index(drop=True)
        y = pd.Series(y).reset_index(drop=True)

        cat_cols = X.select_dtypes(include='object').columns.tolist()
        if cat_cols:
            self.encoder = BayesianOrdinalEncoder(cols=cat_cols)
            X = self.encoder.fit_transform(X, y)

        self.feature_names_ = X.columns.tolist()

        X = pd.DataFrame(self.scaler.fit_transform(X), columns=X.columns)

        F = np.zeros(len(y))
        self.trees = []
        self.tree_weights = []

        for i in range(self.n_estimators):
            p = 1 / (1 + np.exp(-F))
            grad = y - p
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, grad)

            pred = tree.predict(X)
            delta = self.learning_rate * pred
            F += delta

            ll = log_loss(y, 1 / (1 + np.exp(-F)))
            weight = 1 / (ll + 1e-5)
            self.trees.append(tree)
            self.tree_weights.append(weight)

        stacked_preds = self._get_tree_outputs(X)
        if self.stack_model is None:
            from catboost import CatBoostClassifier
            self.stack_model = CatBoostClassifier(
                iterations=50,
                learning_rate=0.1,
                depth=4,
                verbose=0,
                random_seed=42,
                task_type='GPU' if self.fast_mode else 'CPU'
            )
        # Fit le stack model une seule fois
        self.stack_model.fit(stacked_preds, y)

        # Calibration des probabilités : "cv='prefit'" car CatBoost est déjà entraîné
        # self.stack_model = CalibratedClassifierCV(self.stack_model, method="sigmoid", cv="prefit")

        # # Fit du calibrateur uniquement (pas le modèle CatBoost)
        # self.stack_model.fit(stacked_preds, y)

    def _get_tree_outputs(self, X):
        outputs = [tree.predict(X) * w for tree, w in zip(self.trees, self.tree_weights)]
        return np.vstack(outputs).T

    def predict_proba(self, X):
        X = pd.DataFrame(X).reset_index(drop=True)
        X = self._preprocess(X)
        if self.encoder:
            X = self.encoder.transform(X)

        X = pd.DataFrame(self.scaler.transform(X), columns=self.feature_names_)
        stacked_preds = self._get_tree_outputs(X)
        return self.stack_model.predict_proba(stacked_preds)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def preprocess_data(df, features):
    df_subset = df[features].copy()

    all_ordinal_cols = ['full_date', 'lib_jour', 'lib_mois', 'Birth_date', 'Activation_Date',
                        'First_Call_Date', 'Last_Call_Date', 'status_date', 'Document_Validation_Date',
                        'DOC_SCN_DT']
    ordinal_cols = [col for col in all_ordinal_cols if col in df_subset.columns]
    num_cols = df_subset.select_dtypes(include=np.number).columns.tolist()
    cat_cols = df_subset.select_dtypes(include='object').columns.tolist()
    counter_cols = [col for col in cat_cols if col not in ordinal_cols]

    # Pipeline numérique
    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    df_num = pd.DataFrame(num_pipe.fit_transform(df_subset[num_cols]), columns=num_cols, index=df_subset.index).astype(np.float32)

    # Pipeline ordinal
    if ordinal_cols:
        ordinal_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ])
        df_ord = pd.DataFrame(ordinal_pipe.fit_transform(df_subset[ordinal_cols]), columns=ordinal_cols, index=df_subset.index).astype(np.float32)
    else:
        df_ord = pd.DataFrame(index=df_subset.index)

    # Pipeline count encoding
    if counter_cols:
        count_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', ce.CountEncoder())
        ])
        df_count = pd.DataFrame(count_pipe.fit_transform(df_subset[counter_cols]), columns=counter_cols, index=df_subset.index)
        df_count = np.log1p(df_count).astype(np.float32)
    else:
        df_count = pd.DataFrame(index=df_subset.index)

    df_processed = pd.concat([df_num, df_ord, df_count], axis=1)

    # Nettoyage final
    df_processed.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_processed.fillna(0, inplace=True)

    return df_processed.values


def compute_sse(data, centers, labels):
    return np.sum((data - centers[labels]) ** 2)

def noise_based_kmeans(X, max_iter=50, rmin=0.1, rmax=0.5, K_range=(2, 10)):
    N, D = X.shape
    if N < 2:
        raise ValueError("Not enough data points for clustering.")

    best_sse = float('inf')
    best_labels = None
    best_centers = None

    NS = max(1, int(np.sqrt(N) * max_iter))  # Avoid zero
    sqrtN = max(1, int(np.sqrt(N)))
    restart = max(1, max_iter // sqrtN)
    decrease = (rmax - rmin) / max(1, (max_iter - 1))
    noise_radius = rmax

    for iteration in range(max_iter):
        K = random.randint(*K_range)
        if K >= N:
            K = N - 1  # Avoid having more clusters than data points

        init_indices = np.random.choice(range(N), K, replace=False)
        centers = X[init_indices]

        noise = np.random.normal(loc=0.0, scale=noise_radius, size=centers.shape)
        centers += noise

        distances = euclidean_distances(X, centers)
        labels = np.argmin(distances, axis=1)

        new_centers = []
        for i in range(K):
            cluster_points = X[labels == i]
            if len(cluster_points) == 0:
                # Fallback: reinitialize center if cluster is empty
                new_centers.append(X[random.randint(0, N - 1)])
            else:
                new_centers.append(cluster_points.mean(axis=0))
        new_centers = np.array(new_centers)

        sse = compute_sse(X, new_centers, labels)

        if sse < best_sse:
            best_sse = sse
            best_labels = labels
            best_centers = new_centers

        noise_radius = max(noise_radius - decrease, rmin)
        if iteration % restart == 0:
            noise_radius = rmax

    return best_labels, best_centers, best_sse



def visualize_clusters(X, labels, title="Cluster Visualization"):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)

    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='tab10', alpha=0.7)
    plt.title(title)
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.colorbar(scatter, label='Cluster')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

'''# Example Usage
for name, features in feature_sets.items():
    try:
        print(f"\nRunning on {name}")
        X = preprocess_data(df, features)
        labels, centers, sse = noise_based_kmeans(X)
        silhouette = silhouette_score(X, labels)
        print(f"Silhouette Score: {silhouette:.4f}, SSE: {sse:.2f}, Clusters: {len(set(labels))}")
        
        # Show cluster plot
        visualize_clusters(X, labels, title=f"Clusters for {name}")

    except Exception as e:
        print(f"Failed on {name}: {e}")'''
        

'# Example Usage\nfor name, features in feature_sets.items():\n    try:\n        print(f"\nRunning on {name}")\n        X = preprocess_data(df, features)\n        labels, centers, sse = noise_based_kmeans(X)\n        silhouette = silhouette_score(X, labels)\n        print(f"Silhouette Score: {silhouette:.4f}, SSE: {sse:.2f}, Clusters: {len(set(labels))}")\n        \n        # Show cluster plot\n        visualize_clusters(X, labels, title=f"Clusters for {name}")\n\n    except Exception as e:\n        print(f"Failed on {name}: {e}")'

In [46]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from tqdm import tqdm
from category_encoders import TargetEncoder

class FeatureSelectorForAnomaly:
    def __init__(self, n_features=40, corr_threshold=0.85):
        self.n_features = n_features
        self.corr_threshold = corr_threshold
        self.selected_features_stage1 = []
        self.selected_features_final = []
        self.preprocessor = None
        self.all_features = []

    def _prepare_data(self, df):
        df = df.copy()

        all_ordinal_cols = ['full_date', 'lib_jour', 'lib_mois', 'Birth_date', 'Activation_Date',
                            'First_Call_Date', 'Last_Call_Date', 'status_date', 'Document_Validation_Date',
                            'DOC_SCN_DT']
        ordinal_cols = [col for col in all_ordinal_cols if col in df.columns]
        num_cols = df.select_dtypes(include=np.number).columns.tolist()
        cat_cols = df.select_dtypes(include=['object', 'datetime64[ns]']).columns.tolist()
        counter_cols = [col for col in cat_cols if col not in ordinal_cols]

        # Pipeline numérique
        num_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
        df_num = pd.DataFrame(num_pipe.fit_transform(df[num_cols]), columns=num_cols, index=df.index).astype(np.float32)

        # Pipeline ordinal
        if ordinal_cols:
            ordinal_pipe = Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
            ])
            df_ord = pd.DataFrame(ordinal_pipe.fit_transform(df[ordinal_cols]), columns=ordinal_cols, index=df.index).astype(np.float32)
        else:
            df_ord = pd.DataFrame(index=df.index)

        # Pipeline count encoding
        if counter_cols:
            count_pipe = Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', ce.CountEncoder())
            ])
            df_count = pd.DataFrame(count_pipe.fit_transform(df[counter_cols]), columns=counter_cols, index=df.index)
            df_count = np.log1p(df_count).astype(np.float32)
        else:
            df_count = pd.DataFrame(index=df.index)

        df_processed = pd.concat([df_num, df_ord, df_count], axis=1)

        # Nettoyage final
        df_processed.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_processed.fillna(0, inplace=True)

        self.all_features = df_processed.columns.tolist()
        return df_processed.values, self.all_features


    def _remove_correlated(self, X_df, features, threshold=0.85):
        corr_matrix = X_df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
        return [feat for feat in features if feat not in to_drop]

    
    def _unsupervised_feature_scores(self, X, features):
        scores = {}

        # 1. Variance
        variances = np.var(X, axis=0)
        scores['variance'] = variances / variances.max()

        # 2. Isolation Forest scratch
        iso = IsolationForestScratch(n_trees=100)
        iso.fit(X)
        scores['isoforest'] = np.abs(iso.anomaly_score(X)).mean(axis=0)

        # 3. LOF + PCA
        pca = PCA(n_components=min(10, X.shape[1]))
        X_pca = pca.fit_transform(X)
        
        lof = LOF(k=20)
        lof.fit(X_pca)
        lof_scores = lof.anomaly_score()

        # Seuil basé sur le 90e percentile
        labels = (lof_scores > np.percentile(lof_scores, 90)).astype(int)

        scores['pca_info'] = pd.Series(
            mutual_info_classif(X, labels, discrete_features=False),
            index=features
        )

        all_scores = pd.DataFrame(scores, index=features)
        final_score = all_scores.mean(axis=1)
        return final_score.sort_values(ascending=False)

    
    def select_top_features(self, df):
        print("[Étape 1] Préparation des données...")
        X, features = self._prepare_data(df)
        X_df = pd.DataFrame(X, columns=features)

        print("[Étape 2] Calcul des scores de features non supervisés...")
        score_series = self._unsupervised_feature_scores(X, features)

        print("[Étape 3] Sélection des", self.n_features, "meilleures features brutes...")
        top_feats = score_series.head(self.n_features).index.tolist()

        print("[Étape 4] Suppression des features corrélées (>", self.corr_threshold, ")...")
        reduced_feats = self._remove_correlated(X_df[top_feats], top_feats, threshold=self.corr_threshold)
        self.selected_features_stage1 = reduced_feats[:self.n_features]
        print("[Résultat] Features sélectionnées (étape 1) :", len(self.selected_features_stage1))
        return self.selected_features_stage1
        

    def _evaluate_who(self, X, individual):
        selected = np.where(individual == 1)[0]
        if len(selected) < 2:
            return np.inf  # Penalize small subsets

        subset = X[:, selected]
        try:
            # 1. IsolationForest-based clustering
            iso_labels = IsolationForest(random_state=0).fit_predict(subset)
            if len(np.unique(iso_labels)) < 2:
                return np.inf  # Penalize single-cluster solutions
            sil_iso = silhouette_score(subset, iso_labels)

            # 2. Noise-based KMeans clustering
            nbk_labels, _, _ = noise_based_kmeans(subset)
            if len(np.unique(nbk_labels)) < 2:
                return np.inf
            sil_nbk = silhouette_score(subset, nbk_labels)

            # Combine the two silhouette scores
            avg_sil = (sil_iso + sil_nbk) / 2.0

            return -avg_sil + 0.001 * len(selected)  # Minimize loss = -score + complexity_penalty

        except Exception as e:
            return np.inf


    def optimize_with_who(self, X, n_horses=40, n_generations=20, n_stallions=5, mutation_rate=0.1):
        print("[WHO] Optimisation via Wild Horse Optimizer")
        feature_indices = [self.all_features.index(f) for f in self.selected_features_stage1]
        n_features = len(feature_indices)
        # Convert to numpy array first if X is a DataFrame
        if isinstance(X, pd.DataFrame):
            X_sel = X.iloc[:, feature_indices].values.astype(np.float32)
        else:
            X_sel = X[:, feature_indices].astype(np.float32)

        # 1. Initialize population (horses)
        population = np.random.binomial(1, 0.5, size=(n_horses, n_features))
        
        # 2. Evaluate initial population
        fitness = np.array([self._evaluate_who(X_sel, ind) for ind in population])
        best_idx = np.argmin(fitness)
        best_solution = population[best_idx].copy()
        best_score = fitness[best_idx]

        for gen in tqdm(range(n_generations), desc="WHO Generations"):
            # 3. Select stallions (best solutions)
            stallion_indices = np.argsort(fitness)[:n_stallions]
            stallions = population[stallion_indices]

            # 4. Group into harems (assign each horse to a stallion)
            harem_sizes = np.random.randint(1, n_horses - n_stallions + 1, size=n_stallions)
            harem_sizes = (harem_sizes / harem_sizes.sum() * (n_horses - n_stallions)).astype(int)
            harem_sizes[-1] = n_horses - n_stallions - harem_sizes[:-1].sum()  # Adjust last group

            # 5. Grazing behavior (exploration)
            new_population = []
            for i, stallion in enumerate(stallions):
                harem_size = harem_sizes[i]
                harem = np.random.binomial(1, 0.5, size=(harem_size, n_features))  # Random exploration
                harem = np.where(np.random.rand(harem_size, n_features) < 0.3, stallion, harem)  # Follow stallion
                new_population.append(harem)
            
            new_population = np.vstack([stallions] + new_population)
            
            # 6. Breeding (crossover between stallions)
            for i in range(1, n_stallions):
                crossover_point = np.random.randint(1, n_features)
                new_population[i, :crossover_point] = new_population[0, :crossover_point]  # Alpha stallion influence

            # 7. Mutation
            mutation_mask = np.random.rand(n_horses, n_features) < mutation_rate
            new_population = np.where(mutation_mask, 1 - new_population, new_population)

            # 8. Update population and fitness
            population = new_population
            fitness = np.array([self._evaluate_who(X_sel, ind) for ind in population])
            
            # 9. Track best solution
            current_best_idx = np.argmin(fitness)
            if fitness[current_best_idx] < best_score:
                best_solution = population[current_best_idx].copy()
                best_score = fitness[current_best_idx]
            print(f"[WHO] {len(self.selected_features_final)} features selected | Best score: {best_score:.4f}")
        self.selected_features_final = [self.selected_features_stage1[i] for i in range(n_features) if best_solution[i] == 1]
        print(f"[WHO] {len(self.selected_features_final)} features selected | Best score: {best_score:.4f}")
        return self.selected_features_final

    
    def transform_final(self, df):
        # Use the same preprocessing as in _prepare_data
        X_processed, _ = self._prepare_data(df)
        
        # Get indices of selected features
        feat_idx = [self.all_features.index(f) for f in self.selected_features_final]
        
        # Return only the selected columns
        return X_processed[:, feat_idx]

In [ ]:
# Exemple d'utilisation
selector = FeatureSelectorForAnomaly(n_features=40)
selected_stage1 = selector.select_top_features(df)

print("Top features sélectionnées sans corrélation :", selected_stage1)

# WHO sur les 40
X_all, features = selector._prepare_data(df)
selected_final = selector.optimize_with_who(X_all)


print("Features finales sélectionnées par WHO :", selected_final)

X_final = selector.transform_final(df)


[Étape 1] Préparation des données...
[Étape 2] Calcul des scores de features non supervisés...
[Étape 3] Sélection des 40 meilleures features brutes...
[Étape 4] Suppression des features corrélées (> 0.85 )...
[Résultat] Features sélectionnées (étape 1) : 36
Top features sélectionnées sans corrélation : ['Last_Call_Date', 'First_Call_Date', 'reactivite_client', 'full_date', 'Activation_Date', 'temps_moyen_appel', 'cust_id', 'pdv_sk', 'id_date', 'DOC_VAL_USR', 'localisation_sk', 'temps_moyen_traitement', 'ARPU_Last_2_Months', 'Revenue_Last_2_Months', 'Document_Validation_3G', 'Document_Validation_2G', 'lib_jour', 'Id_operation', 'PoS_ID', 'ICC', 'Customer_Type', 'Source', 'lib_mois', 'nin', 'Document_type', 'MSISDN', 'minor_ok', 'doc_scan_avant_activation', 'Street', 'age_sub', 'similarity_score_bin', 'Province', 'City', 'ID_junk', 'DOC_SCN_USR', 'Document_Validation_4G']
[WHO] Optimisation via Wild Horse Optimizer


WHO Generations:   5%|▌         | 1/20 [00:56<17:47, 56.16s/it]

[WHO] 0 features selected | Best score: -0.8801


WHO Generations:  10%|█         | 2/20 [01:52<16:49, 56.07s/it]

[WHO] 0 features selected | Best score: -0.9910


...

[WHO] 0 features selected | Best score: -0.9910


WHO Generations:  55%|█████▌    | 11/20 [11:00<09:53, 65.95s/it]

[WHO] 0 features selected | Best score: -0.9910


WHO Generations:  60%|██████    | 12/20 [11:57<08:24, 63.09s/it]

[WHO] 0 features selected | Best score: -0.9910


In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from tqdm import tqdm
from category_encoders import TargetEncoder

class FeatureSelectorForAnomaly:
    def __init__(self, n_features=40):
        self.n_features = n_features
        self.selected_features_stage1 = []
        self.selected_features_final = []
        self.preprocessor = None
        self.all_features = []

    def _prepare_data(self, df):
        df = df.copy()

        all_ordinal_cols = ['full_date', 'lib_jour', 'lib_mois', 'Birth_date', 'Activation_Date',
                            'First_Call_Date', 'Last_Call_Date', 'status_date', 'Document_Validation_Date',
                            'DOC_SCN_DT']
        ordinal_cols = [col for col in all_ordinal_cols if col in df.columns]
        num_cols = df.select_dtypes(include=np.number).columns.tolist()
        cat_cols = df.select_dtypes(include='object').columns.tolist()
        counter_cols = [col for col in cat_cols if col not in ordinal_cols]

        # Pipeline numérique
        num_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
        df_num = pd.DataFrame(num_pipe.fit_transform(df[num_cols]), columns=num_cols, index=df.index).astype(np.float32)

        # Pipeline ordinal
        if ordinal_cols:
            ordinal_pipe = Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
            ])
            df_ord = pd.DataFrame(ordinal_pipe.fit_transform(df[ordinal_cols]), columns=ordinal_cols, index=df.index).astype(np.float32)
        else:
            df_ord = pd.DataFrame(index=df.index)

        # Pipeline count encoding
        if counter_cols:
            count_pipe = Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', ce.CountEncoder())
            ])
            df_count = pd.DataFrame(count_pipe.fit_transform(df[counter_cols]), columns=counter_cols, index=df.index)
            df_count = np.log1p(df_count).astype(np.float32)
        else:
            df_count = pd.DataFrame(index=df.index)

        df_processed = pd.concat([df_num, df_ord, df_count], axis=1)

        # Nettoyage final
        df_processed.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_processed.fillna(0, inplace=True)

        self.all_features = df_processed.columns.tolist()
        return df_processed.values, self.all_features


    # def _remove_correlated(self, X_df, features, threshold=0.85):
    #     corr_matrix = X_df.corr().abs()
    #     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    #     to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
    #     return [feat for feat in features if feat not in to_drop]

    
    def _unsupervised_feature_scores(self, X, features):
        scores = {}

        # 1. Variance
        variances = np.var(X, axis=0)
        scores['variance'] = variances / variances.max()

        # 2. Isolation Forest scratch
        iso = IsolationForestScratch(n_trees=100)
        iso.fit(X)
        scores['isoforest'] = np.abs(iso.anomaly_score(X)).mean(axis=0)

        # 3. LOF + PCA
        pca = PCA(n_components=min(10, X.shape[1]))
        X_pca = pca.fit_transform(X)
        
        lof = LOF(k=20)
        lof.fit(X_pca)
        lof_scores = lof.anomaly_score()

        # Seuil basé sur le 90e percentile
        labels = (lof_scores > np.percentile(lof_scores, 90)).astype(int)

        scores['pca_info'] = pd.Series(
            mutual_info_classif(X, labels, discrete_features=False),
            index=features
        )

        all_scores = pd.DataFrame(scores, index=features)
        final_score = all_scores.mean(axis=1)
        return final_score.sort_values(ascending=False)

    
    def select_top_features(self, df):
        print("[Étape 1] Préparation des données...")
        X, features = self._prepare_data(df)
        X_df = pd.DataFrame(X, columns=features)

        print("[Étape 2] Calcul des scores de features non supervisés...")
        score_series = self._unsupervised_feature_scores(X, features)

        print("[Étape 3] Sélection des", self.n_features, "meilleures features brutes...")
        top_feats = score_series.head(self.n_features).index.tolist()

        # print("[Étape 4] Suppression des features corrélées (>", self.corr_threshold, ")...")
        # reduced_feats = self._remove_correlated(X_df[top_feats], top_feats, threshold=self.corr_threshold)
        self.selected_features_stage1 = top_feats[:self.n_features]
        print("[Résultat] Features sélectionnées (étape 1) :", len(self.selected_features_stage1))
        return self.selected_features_stage1
        

    def _evaluate_who(self, X, individual):
        selected = np.where(individual == 1)[0]
        if len(selected) < 2:
            return np.inf  # Penalize small subsets

        subset = X[:, selected]
        try:
            # 1. IsolationForest-based clustering
            iso_labels = IsolationForest(random_state=0).fit_predict(subset)
            if len(np.unique(iso_labels)) < 2:
                return np.inf  # Penalize single-cluster solutions
            sil_iso = silhouette_score(subset, iso_labels)

            # 2. Noise-based KMeans clustering
            nbk_labels, _, _ = noise_based_kmeans(subset)
            if len(np.unique(nbk_labels)) < 2:
                return np.inf
            sil_nbk = silhouette_score(subset, nbk_labels)

            # Combine the two silhouette scores
            avg_sil = (sil_iso + sil_nbk) / 2.0

            return -avg_sil + 0.001 * len(selected)  # Minimize loss = -score + complexity_penalty

        except Exception as e:
            return np.inf


    def optimize_with_who(self, X, n_horses=40, n_generations=20, n_stallions=5, mutation_rate=0.1):
        print("[WHO] Optimisation via Wild Horse Optimizer")
        feature_indices = [self.all_features.index(f) for f in self.selected_features_stage1]
        n_features = len(feature_indices)
        # Convert to numpy array first if X is a DataFrame
        if isinstance(X, pd.DataFrame):
            X_sel = X.iloc[:, feature_indices].values.astype(np.float32)
        else:
            X_sel = X[:, feature_indices].astype(np.float32)

        # 1. Initialize population (horses)
        population = np.random.binomial(1, 0.5, size=(n_horses, n_features))
        
        # 2. Evaluate initial population
        fitness = np.array([self._evaluate_who(X_sel, ind) for ind in population])
        best_idx = np.argmin(fitness)
        best_solution = population[best_idx].copy()
        best_score = fitness[best_idx]

        for gen in tqdm(range(n_generations), desc="WHO Generations"):
            # 3. Select stallions (best solutions)
            stallion_indices = np.argsort(fitness)[:n_stallions]
            stallions = population[stallion_indices]

            # 4. Group into harems (assign each horse to a stallion)
            harem_sizes = np.random.randint(1, n_horses - n_stallions + 1, size=n_stallions)
            harem_sizes = (harem_sizes / harem_sizes.sum() * (n_horses - n_stallions)).astype(int)
            harem_sizes[-1] = n_horses - n_stallions - harem_sizes[:-1].sum()  # Adjust last group

            # 5. Grazing behavior (exploration)
            new_population = []
            for i, stallion in enumerate(stallions):
                harem_size = harem_sizes[i]
                harem = np.random.binomial(1, 0.5, size=(harem_size, n_features))  # Random exploration
                harem = np.where(np.random.rand(harem_size, n_features) < 0.3, stallion, harem)  # Follow stallion
                new_population.append(harem)
            
            new_population = np.vstack([stallions] + new_population)
            
            # 6. Breeding (crossover between stallions)
            for i in range(1, n_stallions):
                crossover_point = np.random.randint(1, n_features)
                new_population[i, :crossover_point] = new_population[0, :crossover_point]  # Alpha stallion influence

            # 7. Mutation
            mutation_mask = np.random.rand(n_horses, n_features) < mutation_rate
            new_population = np.where(mutation_mask, 1 - new_population, new_population)

            # 8. Update population and fitness
            population = new_population
            fitness = np.array([self._evaluate_who(X_sel, ind) for ind in population])
            
            # 9. Track best solution
            current_best_idx = np.argmin(fitness)
            if fitness[current_best_idx] < best_score:
                best_solution = population[current_best_idx].copy()
                best_score = fitness[current_best_idx]
            print(f"[WHO] {len(self.selected_features_final)} features selected | Best score: {best_score:.4f}")
        self.selected_features_final = [self.selected_features_stage1[i] for i in range(n_features) if best_solution[i] == 1]
        print(f"[WHO] {len(self.selected_features_final)} features selected | Best score: {best_score:.4f}")
        return self.selected_features_final, best_score

    
    def transform_final(self, df):
        # Use the same preprocessing as in _prepare_data
        X_processed, _ = self._prepare_data(df)
        
        # Get indices of selected features
        feat_idx = [self.all_features.index(f) for f in self.selected_features_final]
        
        # Return only the selected columns
        return X_processed[:, feat_idx]

In [ ]:
# Exemple d'utilisation
selector = FeatureSelectorForAnomaly(n_features=40)
selected_stage1 = selector.select_top_features(df)

print("Top features sélectionnées sans corrélation :", selected_stage1)

# WHO sur les 40
X_all, features = selector._prepare_data(df)
selected_final = selector.optimize_with_who(X_all)


print("Features finales sélectionnées par WHO :", selected_final)

X_final = selector.transform_final(df)


[Étape 1] Préparation des données...
[Étape 2] Calcul des scores de features non supervisés...
[Étape 3] Sélection des 40 meilleures features brutes...
[Résultat] Features sélectionnées (étape 1) : 40
Top features sélectionnées sans corrélation : ['Last_Call_Date', 'First_Call_Date', 'reactivite_client', 'Document_Validation_Date', 'DOC_SCN_DT', 'full_date', 'Activation_Date', 'temps_moyen_appel', 'cust_id', 'pdv_sk', 'id_date', 'lib_jour', 'Document_type', 'Document_Validation_2G', 'jours', 'rejection_reason', 'mois', 'NIN_ok', 'Street', 'taux_rejet', 'temps_moyen_traitement', 'ID_junk', 'localisation_sk', 'SIM_Type', 'nb_total_contrat_traiter', 'Connection_Type', 'Gender', 'taux_validite', 'MSISDN', 'Termination_Date', 'Province', 'nin', 'Status_Date', 'Revenue_Last_2_Months', 'Status', 'ARPU_Last_2_Months', 'Document_Validation_3G', 'Document_Stamped', 'type_d_opeartion', 'PoS_ID']
[WHO] Optimisation via Wild Horse Optimizer


WHO Generations:   5%|▌         | 1/20 [01:34<29:59, 94.72s/it]

[WHO] 0 features selected | Best score: -0.4634


WHO Generations:  10%|█         | 2/20 [03:02<27:13, 90.75s/it]

[WHO] 0 features selected | Best score: -0.4634


...

WHO Generations:  95%|█████████▌| 19/20 [24:02<00:58, 58.90s/it]

[WHO] 0 features selected | Best score: -0.9433


WHO Generations: 100%|██████████| 20/20 [25:41<00:00, 77.06s/it]

[WHO] 0 features selected | Best score: -0.9433
[WHO] 14 features selected | Best score: -0.9433


ValueError: too many values to unpack (expected 3)

In [ ]:
from joblib import Parallel, delayed
import itertools
import pandas as pd
import time
import traceback

def run_experiment(params):
    try:
        n_feat, gen, mut_rate, n_horses, n_stallions = params
        print(f"▶️ Running: n_features={n_feat}, n_horses={n_horses}, generations={gen}, n_stallions={n_stallions}, mutation_rate={mut_rate}")
        
        # Recharge les données dans chaque worker (évite les problèmes de mémoire partagée)
        # df = pd.read_csv("your_data.csv")  # Assurez-vous que le chemin est correct
        
        selector = FeatureSelectorForAnomaly(n_features=n_feat)
        stage1_feats = selector.select_top_features(df)
        X_all, features = selector._prepare_data(df)
        

        start_time = time.time()
        final_feats, score = selector.optimize_with_who(
            X_all, n_horses=n_horses, n_generations=gen,
            n_stallions=n_stallions, mutation_rate=mut_rate
        )
        duration = time.time() - start_time

        return {
            'n_features': n_feat,
            'n_horses': n_horses,
            'generations': gen,
            'n_stallions': n_stallions,
            'mutation_rate': mut_rate,
            'stage1_selected': len(stage1_feats),
            'final_selected': len(final_feats),
            'selected_features': list(final_feats),
            'score': float(score),
            'duration_sec': round(duration, 2)
        }

    except Exception as e:
        print(f"[❌ Erreur] {e}")
        traceback.print_exc()
        return {
            'n_features': n_feat,
            'n_horses': n_horses,
            'generations': gen,
            'n_stallions': n_stallions,
            'mutation_rate': mut_rate,
            'stage1_selected': 0,
            'final_selected': 0,
            'selected_features': [],
            'score': 'error',
            'duration_sec': 0,
            'error': str(e)
        }

if __name__ == '__main__':
    n_features_list = [20, 30, 40]
    n_generations_list = [10, 20, 30]
    mutation_rates = [0.05, 0.1]
    n_horses_list = [20, 30, 40]
    n_stallions_list = [3, 5, 7]

    param_combinations = list(itertools.product(
        n_features_list, n_generations_list, mutation_rates,
        n_horses_list, n_stallions_list
    ))

    print(f"🧠 Lancement de {len(param_combinations)} combinaisons avec joblib...")

    # Utilisation de joblib.Parallel
    results = Parallel(n_jobs=-1, verbose=10)(  # n_jobs=-1 utilise tous les cœurs
        delayed(run_experiment)(params) for params in param_combinations
    )

    df_results = pd.DataFrame(results)
    df_results.to_csv('feature_selector_experiments_joblib.csv', index=False)
    print("✅ Résultats enregistrés dans 'feature_selector_experiments_joblib.csv'")

🧠 Lancement de 162 combinaisons avec joblib...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 29.1min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 42.6min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 80.8min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 135.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 226.8min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed: 338.3min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed: 396.7min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed: 447.6min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed: 592.3min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed: 694.2min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed: 767.9min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 849.0min
[Parallel(n_jobs=-1)]: Done 156 out of 162 | elapsed: 971.2min remaining: 37.4min


✅ Résultats enregistrés dans 'feature_selector_experiments_joblib.csv'


[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed: 1004.9min finished


PSO


In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from tqdm import tqdm
from category_encoders import TargetEncoder

class FeatureSelectorForAnomaly:
    def __init__(self, n_features=40):
        self.n_features = n_features
        self.selected_features_stage1 = []
        self.selected_features_final = []
        self.preprocessor = None
        self.all_features = []

    def _prepare_data(self, df):
        df = df.copy()

        all_ordinal_cols = ['full_date', 'lib_jour', 'lib_mois', 'Birth_date', 'Activation_Date',
                            'First_Call_Date', 'Last_Call_Date', 'status_date', 'Document_Validation_Date',
                            'DOC_SCN_DT']
        ordinal_cols = [col for col in all_ordinal_cols if col in df.columns]
        num_cols = df.select_dtypes(include=np.number).columns.tolist()
        cat_cols = df.select_dtypes(include='object').columns.tolist()
        counter_cols = [col for col in cat_cols if col not in ordinal_cols]

        # Pipeline numérique
        num_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
        df_num = pd.DataFrame(num_pipe.fit_transform(df[num_cols]), columns=num_cols, index=df.index).astype(np.float32)

        # Pipeline ordinal
        if ordinal_cols:
            ordinal_pipe = Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
            ])
            df_ord = pd.DataFrame(ordinal_pipe.fit_transform(df[ordinal_cols]), columns=ordinal_cols, index=df.index).astype(np.float32)
        else:
            df_ord = pd.DataFrame(index=df.index)

        # Pipeline count encoding
        if counter_cols:
            count_pipe = Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', ce.CountEncoder())
            ])
            df_count = pd.DataFrame(count_pipe.fit_transform(df[counter_cols]), columns=counter_cols, index=df.index)
            df_count = np.log1p(df_count).astype(np.float32)
        else:
            df_count = pd.DataFrame(index=df.index)

        df_processed = pd.concat([df_num, df_ord, df_count], axis=1)

        # Nettoyage final
        df_processed.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_processed.fillna(0, inplace=True)

        self.all_features = df_processed.columns.tolist()
        return df_processed.values, self.all_features


    # def _remove_correlated(self, X_df, features, threshold=0.85):
    #     corr_matrix = X_df.corr().abs()
    #     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    #     to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
    #     return [feat for feat in features if feat not in to_drop]

    
    def _unsupervised_feature_scores(self, X, features):
        scores = {}

        # 1. Variance
        variances = np.var(X, axis=0)
        scores['variance'] = variances / variances.max()

        # 2. Isolation Forest scratch
        iso = IsolationForestScratch(n_trees=100)
        iso.fit(X)
        scores['isoforest'] = np.abs(iso.anomaly_score(X)).mean(axis=0)

        # 3. LOF + PCA
        pca = PCA(n_components=min(10, X.shape[1]))
        X_pca = pca.fit_transform(X)
        
        lof = LOF(k=20)
        lof.fit(X_pca)
        lof_scores = lof.anomaly_score()

        # Seuil basé sur le 90e percentile
        labels = (lof_scores > np.percentile(lof_scores, 90)).astype(int)

        scores['pca_info'] = pd.Series(
            mutual_info_classif(X, labels, discrete_features=False),
            index=features
        )

        all_scores = pd.DataFrame(scores, index=features)
        final_score = all_scores.mean(axis=1)
        return final_score.sort_values(ascending=False)

    
    def select_top_features(self, df):
        print("[Étape 1] Préparation des données...")
        X, features = self._prepare_data(df)
        X_df = pd.DataFrame(X, columns=features)

        print("[Étape 2] Calcul des scores de features non supervisés...")
        score_series = self._unsupervised_feature_scores(X, features)

        print("[Étape 3] Sélection des", self.n_features, "meilleures features brutes...")
        top_feats = score_series.head(self.n_features).index.tolist()

        # print("[Étape 4] Suppression des features corrélées (>", self.corr_threshold, ")...")
        # reduced_feats = self._remove_correlated(X_df[top_feats], top_feats, threshold=self.corr_threshold)
        self.selected_features_stage1 = top_feats[:self.n_features]
        print("[Résultat] Features sélectionnées (étape 1) :", len(self.selected_features_stage1))
        return self.selected_features_stage1
        

    def _evaluate_who(self, X, individual):
        selected = np.where(individual == 1)[0]
        if len(selected) < 2:
            return np.inf  # Penalize small subsets

        subset = X[:, selected]
        try:
            # 1. IsolationForest-based clustering
            iso_labels = IsolationForest(random_state=0).fit_predict(subset)
            if len(np.unique(iso_labels)) < 2:
                return np.inf  # Penalize single-cluster solutions
            sil_iso = silhouette_score(subset, iso_labels)

            # 2. Noise-based KMeans clustering
            nbk_labels, _, _ = noise_based_kmeans(subset)
            if len(np.unique(nbk_labels)) < 2:
                return np.inf
            sil_nbk = silhouette_score(subset, nbk_labels)

            # Combine the two silhouette scores
            avg_sil = (sil_iso + sil_nbk) / 2.0

            return -avg_sil + 0.001 * len(selected)  # Minimize loss = -score + complexity_penalty

        except Exception as e:
            return np.inf

    def optimize_with_pso(self, X, n_particles=40, n_iterations=20, inertia=0.7, cognitive=1.4, social=1.4):
            print("[PSO] Optimisation via Particle Swarm Optimization")
            feature_indices = [self.all_features.index(f) for f in self.selected_features_stage1]
            n_features = len(feature_indices)
            X_np = X.values if isinstance(X, pd.DataFrame) else X
            X_sel = X_np[:, feature_indices].astype(np.float32)


            # Initialize particle positions and velocities
            particles = np.random.rand(n_particles, n_features)
            velocities = np.random.randn(n_particles, n_features) * 0.1
            particles = (particles > 0.5).astype(int)

            personal_best = particles.copy()
            personal_best_scores = np.array([self._evaluate_who(X_sel, p) for p in particles])

            global_best = personal_best[np.argmin(personal_best_scores)].copy()
            global_best_score = np.min(personal_best_scores)

            for _ in tqdm(range(n_iterations), desc="PSO Iterations"):
                for i in range(n_particles):
                    r1, r2 = np.random.rand(), np.random.rand()
                    velocities[i] = (
                        inertia * velocities[i] +
                        cognitive * r1 * (personal_best[i] - particles[i]) +
                        social * r2 * (global_best - particles[i])
                    )
                    sigmoid = 1 / (1 + np.exp(-velocities[i]))
                    particles[i] = (np.random.rand(n_features) < sigmoid).astype(int)

                    score = self._evaluate_who(X_sel, particles[i])
                    if score < personal_best_scores[i]:
                        personal_best[i] = particles[i].copy()
                        personal_best_scores[i] = score
                        if score < global_best_score:
                            global_best = particles[i].copy()
                            global_best_score = score
                print(f"[PSO] {len(self.selected_features_final)} features selected | Best score: {global_best_score:.4f}")
            self.selected_features_final = [
                self.selected_features_stage1[i]
                for i in range(n_features) if global_best[i] == 1
            ]
            print(f"[PSO] {len(self.selected_features_final)} features selected | Best score: {global_best_score:.4f}")
            return self.selected_features_final

    def transform_final(self, df):
        # Use the same preprocessing as in _prepare_data
        X_processed, _ = self._prepare_data(df)
        
        # Get indices of selected features
        feat_idx = [self.all_features.index(f) for f in self.selected_features_final]
        
        # Return only the selected columns
        return X_processed[:, feat_idx]

In [7]:
# Exemple d'utilisation
selector = FeatureSelectorForAnomaly(n_features=40)
selected_stage1 = selector.select_top_features(df)

print("Top features sélectionnées sans corrélation :", selected_stage1)

# WHO sur les 40
X_all, features = selector._prepare_data(df)
selected_final = selector.optimize_with_pso(X_all)


print("Features finales sélectionnées par PSO :", selected_final)

X_final = selector.transform_final(df)


[Étape 1] Préparation des données...
[Étape 2] Calcul des scores de features non supervisés...
[Étape 3] Sélection des 40 meilleures features brutes...
[Résultat] Features sélectionnées (étape 1) : 40
Top features sélectionnées sans corrélation : ['Birth_date', 'Last_Call_Date', 'First_Call_Date', 'reactivite_client', 'status_date', 'Document_Validation_Date', 'full_date', 'DOC_SCN_DT', 'temps_moyen_appel', 'Activation_Date', 'id_date', 'jours', 'pdv_sk', 'localisation_sk', 'lib_jour', 'temps_moyen_traitement', 'ARPU_Last_2_Months', 'year', 'doc_scan_avant_activation', 'Province', 'PoS_ID', 'Postal_ID', 'Revenue_Last_2_Months', 'Revenue_Last_12_Months', 'DOB_ok', 'tarrif_profile', 'lib_mois', 'ID_doc', 'type_d_operation', 'Street', 'id_type', 'NIN_ok', 'id_operation', 'client_haut_revenue', 'age_sub', 'minor_ok', 'ocr_violation', 'similarity_score', 'similarity_score_bin', 'cn_valid']
[PSO] Optimisation via Particle Swarm Optimization


PSO Iterations:   5%|▌         | 1/20 [00:48<15:15, 48.17s/it]

[PSO] 0 features selected | Best score: -0.3898


PSO Iterations:  10%|█         | 2/20 [01:36<14:26, 48.12s/it]

[PSO] 0 features selected | Best score: -0.3898


...

[PSO] 0 features selected | Best score: -0.4622


PSO Iterations:  95%|█████████▌| 19/20 [19:02<01:10, 70.98s/it]

[PSO] 0 features selected | Best score: -0.4622


PSO Iterations: 100%|██████████| 20/20 [20:24<00:00, 61.21s/it]

[PSO] 0 features selected | Best score: -0.4622
[PSO] 17 features selected | Best score: -0.4622
Features finales sélectionnées par PSO : ['reactivite_client', 'Document_Validation_Date', 'temps_moyen_appel', 'id_date', 'jours', 'localisation_sk', 'lib_jour', 'temps_moyen_traitement', 'doc_scan_avant_activation', 'PoS_ID', 'Revenue_Last_12_Months', 'tarrif_profile', 'type_d_operation', 'id_type', 'NIN_ok', 'id_operation', 'similarity_score']


In [9]:
from joblib import Parallel, delayed
import itertools
import pandas as pd
import time
import traceback

def run_experiment(params):
    try:
        n_feat, n_particles, n_iterations, inertia, cognitive, social= params
        print(f"▶️ Running: n_features={n_feat}, n_particles={n_particles}, n_iterations={n_iterations}, inertia={inertia}, mutation_rate={cognitive}, social={social}")
        
        # Recharge les données dans chaque worker (évite les problèmes de mémoire partagée)
        # df = pd.read_csv("your_data.csv")  # Assurez-vous que le chemin est correct
        
        selector = FeatureSelectorForAnomaly(n_features=n_feat)
        stage1_feats = selector.select_top_features(df)
        X_all, features = selector._prepare_data(df)
        

        start_time = time.time()
        final_feats, score = selector.optimize_with_pso(
            X_all, n_particles=n_particles, n_iterations=n_iterations,
            inertia=inertia, cognitive=cognitive, social=social
        )
        duration = time.time() - start_time

        return {
            'n_features': n_feat,
            'n_particles': n_particles,
            'n_iterations': n_iterations,
            'inertia': inertia,
            'cognitive': cognitive,
            'social': social,
            'stage1_selected': len(stage1_feats),
            'final_selected': len(final_feats),
            'selected_features': list(final_feats),
            'score': float(score),
            'duration_sec': round(duration, 2)
        }

    except Exception as e:
        print(f"[❌ Erreur] {e}")
        traceback.print_exc()
        return {
            'n_features': n_feat,
            'n_particles': n_particles,
            'n_iterations': n_iterations,
            'inertia': inertia,
            'cognitive': cognitive,
            'social': social,
            'stage1_selected': 0,
            'final_selected': 0,
            'selected_features': [],
            'score': 'error',
            'duration_sec': 0,
            'error': str(e)
        }

if __name__ == '__main__':
    n_features_list = [20, 30, 40]
    n_particles = [20, 30, 40]
    n_iterations =  [10, 20, 30]
    inertia = [0.4, 0.7, 0.9]
    cognitive = [1.4, 2.0, 2.5]
    social = [1.4, 2.0, 2.5]

    param_combinations = list(itertools.product(
        n_features_list, n_particles, n_iterations, inertia, cognitive, social
        
    ))

    print(f"🧠 Lancement de {len(param_combinations)} combinaisons avec joblib...")

    # Utilisation de joblib.Parallel
    results = Parallel(n_jobs=-1, verbose=10)(  # n_jobs=-1 utilise tous les cœurs
        delayed(run_experiment)(params) for params in param_combinations
    )

    df_results = pd.DataFrame(results)
    df_results.to_csv('feature_selector_experiments_joblib_pso.csv', index=False)
    print("✅ Résultats enregistrés dans 'feature_selector_experiments_joblib.csv'")

🧠 Lancement de 729 combinaisons avec joblib...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 31.2min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 57.5min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 80.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 122.1min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed: 141.7min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed: 225.3min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed: 288.2min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed: 336.3min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed: 377.5min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed: 490.0min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 589.9min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed: 744.9min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 800.2min
[Parallel(n_jobs=-1)]: Done 197 tasks      | 

✅ Résultats enregistrés dans 'feature_selector_experiments_joblib.csv'
